Cette analyse reprend les résultats d'une expérience de dégustation en double aveugle menée à l'occasion d'une [conférence de l'Association Française pour l'Information Scientifique](https://www.youtube.com/watch?v=gGIKDXt-NRE) sur le thème des vins biodynamiques. Cette expérience a notamment servi à remettre en question les proclamations de certains médias, selon lesquels les vins biodynamiques seraient franchement supérieurs en goût aux vins conventionnels.

On propose ici **un traitement statistique différent de celui fait pendant la conférence**, ce qui peut permettre une interprétation plus intuitive des résultats. (Plus précisément : au lieu de calculer la p-valeur pour un _Test de Signifiance à Hypothèse Nulle,_ nous faisons une _Analyse de Données Bayésienne_.) Ceci nous permet notamment :

- de répondre à des questions comme : _'quelle est la probabilité que le rouge biodynamique soit supérieur au rouge conventionnel ?'_
- de laisser dans l'analyse **une place pour les a-priori personnels,** basés sur le vécu.

Le code source de l'analyse est inclus pour les personnes qui souhaiteraient examiner ou reproduire le traitement statistique.

Pour les autres, nous recommandons d'aller directement aux **conclusions:**

- **[Résumé grand-public](#broad-audience-summary)**
- **[Résumé pour relecteurs scientifiques](#scientific-summary)**

# Code source

In [21]:
import pandas as pd
import scipy.stats as sp_stats
#import statsmodels.api as stm
#import statsmodels.stats.power as smpow

from IPython.display import HTML, display

#### Input data

In [2]:
WINE_WHITE = "vin blanc"
WINE_RED = "vin rouge"
wine_colours = [WINE_WHITE, WINE_RED]

CONV = "conventionnel"
BIOD = "biodynamique"

wine_labels = {
    WINE_WHITE: {BIOD: "A", CONV: "B"},
    WINE_RED: {BIOD: "C", CONV: "D"}
}

## Note this data won't be used in the analysis.
notes_counts = pd.DataFrame(index = [1,2,3,4,5], data={
    "A": [3,6,7,9,3], 
    "B": [2,7,10,7,2], 
    "C": [3,7,7,9,1], 
    "D": [2,3,11,7,4]
  })

In [3]:
notes_counts

,A,B,C,D
1,3,2,3,2
2,6,7,7,3
3,7,10,7,11
4,9,7,9,7
5,3,2,1,4


In [4]:
EX_AEQUO = "pas de préférence"
PREF_BIOD = "préfère biodynamique"
PREF_CONV = "préfère conventionnel"
pref_types = [PREF_CONV, EX_AEQUO, PREF_BIOD]


prefs_float = pd.DataFrame(
    index = [WINE_WHITE, WINE_RED], 
    data = {
        PREF_CONV: [0.43, 0.37],
        EX_AEQUO: [0.14, 0.11],
        PREF_BIOD: [0.43, 0.52]
    })
prefs_float

,préfère conventionnel,pas de préférence,préfère biodynamique
vin blanc,0.43,0.14,0.43
vin rouge,0.37,0.11,0.52


In [5]:
sample_sizes = {
    WINE_WHITE: notes_counts[wine_labels[WINE_WHITE][CONV]].sum(),
    WINE_RED: notes_counts[wine_labels[WINE_RED][CONV]].sum()
}


def guess_pref_counts():
    """
    Works out the how many people had each preference, based on the reported fractions.
    """

    def n_having_pref(wine, pref):
        if (pref == EX_AEQUO):
            return sample_sizes[wine] - (n_having_pref(wine, PREF_CONV) + n_having_pref(wine, PREF_BIOD))
        else:
            fraction = prefs_float[pref][wine]
            sample_size = sample_sizes[wine]
            return int(round(fraction * sample_size))
        
    column_series = {}
    for pref in pref_types:
        col_counts = []
        for wine in wine_colours:
           col_counts.append(n_having_pref(wine, pref))
        column_series[pref] = col_counts
    
    prefs_counts = pd.DataFrame(
        index = wine_colours, 
        data = dict(column_series)
    )
    return prefs_counts


prefs_counts = guess_pref_counts()
prefs_counts

,préfère conventionnel,pas de préférence,préfère biodynamique
vin blanc,12,4,12
vin rouge,10,3,14


#### Bayesian inference

In [16]:
def credibility_that_biodynamical_is_superior(superiority_threshold, prior_counts, pref_counts_for_colour):
    """
    Computes the credibility (i.e posterior probability) that the biodynamical wine is 'superior' (see below) given observed preference counts.
    
    The proposition 'the biodynamical wine is superior' is true if the probability of someone giving a better score
    to the biodynamical wine is at least superiority_threshold.
    
    
    
    Returns a tuple of probabilities: (posterior_credibility, prior_credibility)
    """
    
    def p_biod_sup(pref_counts):
        n_biod = 1 + pref_counts[PREF_BIOD]
        n_non_biod = sum([(1 + pref_counts[pref]) for pref in pref_types if not (pref == PREF_BIOD)])
        bet = sp_stats.beta(n_biod, n_non_biod)
        return bet.sf(superiority_threshold)
    
    return (p_biod_sup(prior_counts + pref_counts_for_colour), p_biod_sup(prior_counts))


superiority_threshold = 2./3.

def make_prior_counts(dict_prefs_counts):
    return pd.Series(data=dict_prefs_counts, index=None)

In [14]:
priors = [{
    'prior_label': "Neutre",
    'prior_description': "Même crédibilité pour toutes les répartitions possibles entre " + "/".join(pref_types),
    'prior_previous_counts': {PREF_BIOD: 0, EX_AEQUO: 0, PREF_CONV:0}
}, {
    'prior_label': "Uniforme sur proportion de préférence biodynamique",
    'prior_description': "Même crédibilité pour toutes les proportions possibles de " + PREF_BIOD,
    'prior_previous_counts': {PREF_BIOD: 0, EX_AEQUO: -0.5, PREF_CONV:-0.5}
}, {
    'prior_label': "Légèrement favorable biodynamique",
    'prior_description': "Correspond à une expérience antérieure de 6 avis préférant le biodynamique à 83%",
    'prior_previous_counts': {PREF_BIOD: 5, EX_AEQUO: 1, PREF_CONV:0}
}, {
    'prior_label': "Assez favorable biodynamique",
    'prior_description': "Correspond à une expérience antérieure de 8 avis préférant le biodynamique à 75%",
    'prior_previous_counts': {PREF_BIOD: 7, EX_AEQUO: 1, PREF_CONV:0}
}, {
    'prior_label': "Très favorable biodynamique",
    'prior_description': "Correspond à une expérience antérieure de 10 avis préférant le biodynamique unanimement.",
    'prior_previous_counts': {PREF_BIOD: 10, EX_AEQUO: 0, PREF_CONV:0}
}]

In [20]:
## Computing an HTML table of the results

def thead(rows):
    ret = '<thead>'
    for row in rows:
        ret += '<tr>'
        for (title, colspan) in row:
            ret += '<th colspan=' + str(colspan) + '>' + title + '</th>'
        ret += '</tr>'
    ret += '</thead>'
    return ret

def html_tr(values):
    ret = ''
    ret += '<tr>'
    for v in values:
        ret += '<td>' + str(v) + '</td>'
    ret += '</tr>'
    return ret
    
        
def results_table():
    table_html = '<table>'
    table_html += thead([[("A priori",1), ("Vins",1), ("Crédibilité de 'biodynamique >> conventionnel'", 2)],
                        [("",1),
                         ("",1),
                         ("Avant expérience",1),
                         ("Après expérience",1),]])
    for prior in priors:
        for wine in wine_colours:
            (p1, p0) = credibility_that_biodynamical_is_superior(superiority_threshold, make_prior_counts(prior['prior_previous_counts']), prefs_counts.loc[wine])
            table_html += html_tr([
                prior['prior_label'],
                wine,
                "{:.2f}".format(p0 * 100) + "%",
                "{:.2f}".format(p1 * 100) + "%"
             ])
    table_html += '</table>'
    return table_html


# Conclusions

<a id="broad-audience-summary"></a>

## Résumé grand-public


Cette étude analyse les résultats de la dégustation de manière à **estimer la proportion des personnes qui donneraient une meilleure note au vin biodynamique.**

Le nombre de voix dans les préférences exprimées en donnent une idée intuitive : 43% pour les vins blancs, 52% pour les vins rouges. Cependant, ces chiffres sont approximatifs, à cause du petit nombre de participants. Peut-être que la vraie proportion est 70%, et que les vins biodynamiques n'ont pas eu de chance.

In [10]:
prefs_counts ## Ci-dessous, les préférences exprimées lors de la dégustation :

,préfère conventionnel,pas de préférence,préfère biodynamique
vin blanc,12,4,12
vin rouge,10,3,14


Nous partons du principe que si le vin biodynamique était aussi nettement supérieur au vin conventionnel que le prétendent certains médias, au moins 2 personnes sur 3 préfèreraient le vin biodynamiques, et donc la vraie proportion serait au dessus de 66% : si on demandait à 100 millions de personnes de faire la dégustation, plus 66 millions préfèreraient le vin biodynamique.

Notre analyse statistique calcule la probabilité que cette proprotion soit si élevée, en tenant compte des a-priori que le lecteur peut raisonnablement avoir de par son expérience préalable des vins biodynamiques. Les résultats sont les suivants :

In [18]:
display(HTML(results_table())) ## Les résultats du traitement statistique :

Ces résultats montrent pour chaque paire de vins que **l'hypothèse selon laquelle le vin biodynamique serait _nettement supérieur_ au goût est peu crédible,** même en partant d'un a-priori personnel très biaisé en faveur du biodynamique.

**Comment interpréter:** La colonne de droite dit par exemple qu'une personne rationnelle avec l'a-priori _'Assez favorable au biodynamique',_ serait avant l'expérience confiante à 70,09% dans la nette supériorité du vin rouge biodynamique, et après l'expérience ne serait plus confiante qu'à 2,46%.

- Une manière intuitive d'interpréter ces pourcentages est la suivante : avant l'expérience, cette personne parierait à 7 contre 3 que le rouge biodynamique est _nettement supérieur._ Puis, en tirant les apprentissages de l'expérience, cette personne changerait d'avis, et parierait à 97 contre 3 que le rouge biodynamique n'est **pas** _nettement supérieur._

Un aspect intéressant de ce type d'analyses statistiques (dites _bayésiennes_ ) est que contrairement aux tests d'hypothèses avec p-valeur, elles prennent en compte les a-priori personnels. Ça peut être intéressant pour s'adresser à des personnes qui revendiquent que leur pressentiment, forgé par leur vécu, se voie accordé le poids qu'il mérite dans l'analyse.

<a id="scientific-summary"></a>

## Résumé pour relecteurs scientifiques

#### Modélisation de l'expérience

On modélise la préférence mesurée des goûteurs comme autant de réalisations indépendantes d'une loi multinomiale à 3 valeurs ("préfère biodynamique" / "préfère conventionnel" / "pas de préférence"). 

- Les 3 paramètres de cette loi correspondent donc aux probabilités qu'une personne prise au hasard préfère respectivement le vin biodynamique, le vin conventionnel, ou n'ait pas de préférence. 
- On cherche à calculer la distribution de probabilité a-posteriori pour ces 3 paramètres, via une inférence Bayésienne.

On propose plusieurs distributions a-priori, plus ou moins favorables au vins biodynamiques.
- Toutes sont sous forme de distributions de Dirichlet: chacune est donc équivalente à avoir assisté à une expérience antérieure de dégustation en double aveugle.
- L'utilisation de distributions de Dirichlet permet aussi une dérivation analytique pratique: notamment, les distributions a-posteriori sont aussi des Distributions de Dirichlet.

#### Résultats rapportés

On définit la proposition *'le vin biodynamique est nettement supérieur au vin conventionnel'* , qu'on note *'biodynamique >> conventionnel'*, par $\mathbb P[\text{préfère biodynamique}] \geq \frac{2}{3}$. C'est donc une condition sur l'un des coefficients de la loi multinomiale mesurée.

- **Justification:** l'analyse se fait dans le cadre d'une remise en question de proclamations dithyrambiques prétendant à la supériorité sans équivoque des vins biodynamqiues, notamment dans les médias. Le ratio de $\frac{2}{3}$ nous semble ne pas mettre la barre très haut pour que permettre à ces proclamations de faire leurs preuves.

On rapportera $\mathbb P[\text{biodynamique >> conventionnel } | \text{ résultats mesurés, distribution a-priori}]$, c'est à dire la crédibilité après expérience que le vin biodynamique dégusté soit 'supérieur' au vin conventionnel dégusté. 

- En notant $\pi_B := \mathbb P[\text{préfère biodynamique}]$, on rapportera donc $\mathbb P[\pi_B \geq \frac{2}{3}]$ a-posteriori.
- On rapportera ce résultat pour chaque paire de vins comparés (rouges et blancs) et chaque distribution a-priori.


#### Dérivation analytique

Les distributions a-priori considérées sont toutes des distributions de Dirichlet. Ces distributions étant conjuguées à la loi multinomiale, les distributions a-posteriori sont aussi de Dirichlet, dont les coefficients sont obtenus en ajoutant le "nombre de votes" pour chaque préférence au coefficient a-priori.

En marginalisant, il vient que la distribution a-posteriori de $\pi_B$ suit une loi Beta. On en déduit la probabilité a-posteriori de l'hypothèse de _nette supériorité._

#### Résultats

Les données mesurées pour chaque paire de vins sont :

In [12]:
prefs_counts

,préfère conventionnel,pas de préférence,préfère biodynamique
vin blanc,12,4,12
vin rouge,10,3,14


In [13]:
display(HTML(results_table()))

Ces résulats indiquent pour chaque paire de vins que **même en partant d'un a-priori fortement biaisé en faveur du vin biodynamique, il est peu crédible que le vin biodynamique soit _nettement supérieur_ au vin conventionnel.**